### MNIST classification

In [1]:
require 'nn';
require 'rnn';
matio = require 'matio'

In [2]:
data = matio.load('ex4data1.mat')
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1 }]

In [3]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);
 
function trainset:size()
    return self.data:size(1)
end

In [4]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [5]:
batchSize = 5000
rho = 5
hiddenSize = 10
nIndex = 20
nClass = 10

In [6]:
rnn = nn.Sequential()
r = nn.Recurrent(
    hiddenSize, nn.Linear(nIndex, hiddenSize),
    nn.Linear(hiddenSize, hiddenSize), nn.Sigmoid(),
    rho
)
rnn:add(r)
rnn:add(nn.Linear(hiddenSize, nClass))
rnn:add(nn.LogSoftMax())
rnn = nn.Sequencer(rnn)

In [7]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion())

### train

In [17]:
lr = 0.1
i = 1

In [20]:
prev = 100
for epoch = 1,1e2 do
    local inputs, targets = {}, {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{},{i, i+nIndex-1}}])
        table.insert(targets, trainset.label[{{}}])
        i = i+20
        if i+nIndex-1 > 400 then 
            i = 1
            break
        end
    end
    local outputs = rnn:forward(inputs)
    local err = criterion:forward(outputs, targets)
    if epoch%1 == 0 then print(epoch, err/rho, i) end
    local gradOutputs = criterion:backward(outputs, targets)
    rnn:backward(inputs, gradOutputs)
    rnn:updateParameters(lr)
    rnn:zeroGradParameters()
    if prev<err and lr > 0.1 then 
        print("prev: ", prev, "cur: ", err, "lr", lr, "->", lr*0.5)
        lr = lr * 0.5 
    end
    prev = err
end

1	2.2623325467748	101	


2	1.8440018558042	201	


3	1.9691901419904	301	


4	2.2418026958367	1	


5	2.2622659944138	101	


...

98	1.7789051946369	201	


99	

1.9192861345837	301	


100	2.2246115778632	

1	


### test

In [21]:
correction = {}
trainsize = 100 -- trainset:size()
for i=20,38 do
    correction[i] = 0
end
for i=1,trainsize do
    local answer = trainset.label[i]
    local inputs = {}
    for step=0,38 do
        table.insert(inputs, trainset.data[{{i},{step*10+1, step*10+20}}])
    end
    local prediction = rnn:forward(inputs)
    for d=20,38 do
        guess = prediction[d][{1,{}}]
        local confidences, indices = torch.max(guess)
        -- if i%100 == 1 then print(answer, guess, indices[1]) end
        if (answer == indices) then
            correction[d] = correction[d] + 1
        end
    end
end
for i=20,38 do
    print(i, " = ", correction[i], 100*correction[i]/trainsize .. '%')
end

20	 = 	18	18%	
21	 = 	2	

2%	
22	 = 	8	8%	
23	 = 	3	3%	
24	 = 	5	5%	
25	 = 	8	8%	
26	 = 	12	12%	
27	 = 	33	33%	
28	 = 	26	26%	
29	 = 	61	61%	
30	 = 	31	31%	
31	 = 	52	52%	
32	 = 	20	20%	
33	 = 	36	36%	
34	 = 	5	5%	
35	 = 	14	14%	
36	 = 	2	2%	
37	 = 	1	1%	
38	 = 	0	0%	
